## Furniture Company -  Version 5

**This version has scaled down values to enhance the experience of the second game. It is also adjusted to provide integer costs, if items should be split into parts.**

Aspects to consider when working with different combinations of variables:

### Variation among items

+ Looking at variation coefficients among the items for every month, a low value indicates that different furniture is built is distributed similarly.
+ At a higher value results indicate the concentration on one item mainly built at one month.

### Variation for each item over a year

+ Higher values indicate, that the months in which these items are built vary more, e.g they are built in just one month throughout the year.
+ Low values show that the distribution of items throughout the year is balanced.
+ This enables for example one item to be mainly built in every month, while other items can be relevant just in some months.

+ Values < 0.5 makes the items appear regularly
+ Values of >= 1 makes them appear in 8 of 12 months with different quantity.

+ A useful criterion would be values smaller than 1.1 and greater than 0.65.

## Model

In [1]:
# This is used to import the modules from the experiment folder
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
# First the model builder functionality needs to be imported.
from model_builder import Item, ModelBuilder
import numpy as np

In [2]:
# These values are needed to build the model
factor_resources = 1
# define the profit per month for each item:

chair_profit =    [2,3,3,2,2,1,2,3,2,2,2,2]
table_profit =    [4,4,4,5,4,3,4,5,4,4,3,3]
bed_profit =      [7,7,5,5,3,5,4,4,5,4,5,6]
bookcase_profit =[7,8,10,8,6,7,7,8,10,9,7,10]

# define the items:
chair = Item(costs_wood=4,costs_metal=1,costs_time_one=4,costs_time_two=1,profit=chair_profit)
table = Item(2,5,4,6,table_profit)
bed = Item(4,3,3,4, bed_profit)
bookcase = Item(5,7,5,3,bookcase_profit)
# Making calculations for one year
months = 12
# specifying available materials
avail_hours_a = 26*factor_resources
avail_hours_b = 30*factor_resources
avail_hours_c = 23*factor_resources
avail_hours_d = 26*factor_resources

avail_wood = list(map(lambda i: round(i/8), [200, 300, 457, 372, 322, 432, 413, 406, 377, 412, 395, 366]))
avail_metal = list(map(lambda i: round(i/8), [463, 168, 773, 380, 353, 391, 392, 473, 370, 350, 351, 293]))

In [ ]:
# Set up the model builder: 
model_builder = ModelBuilder(months=months, avail_wood=avail_wood, avail_metal=avail_metal
                            , avail_hours_a=avail_hours_a,avail_hours_b=avail_hours_b
                            , avail_hours_c=avail_hours_c,avail_hours_d=avail_hours_d
                            , chair=chair,table=table, bed=bed, bookcase=bookcase)
# build the model:
model = model_builder.build_model()


In [ ]:
model_builder.save_model('model_v5_parts')
a.optimal_solution.save_solution('model_v5_solution_parts')

## Random Agent
This agent solves the problem by randomly picking affordable items and producing a random number of the chosen item until nothing more can be produced. 

In [ ]:
# importing a random agent
from agents import RandomAgent
agents = []
a = RandomAgent(model)
a.simulate_year()
agents.append(a)

In [ ]:
#create multiple random agents and simulate the year
# Step through every month with the strategy of the agent
random_agents = []
for i in range(100):
    r = RandomAgent(model)
    r.simulate_year()
    random_agents.append(r)

mean_profits = [int(np.round(np.mean([agent.profits[i] for agent in random_agents]))) for i in range(12)]
print('Mean Profit of random agent: ', sum(mean_profits))
print('Optimal Solution Profit: ', sum(random_agents[0].optimal_solution.profits))

In [ ]:
a.print_profits()

### Plots

In [ ]:
# Plot the profit for each month of the agent compared to the optimal solution
a.plot_profits()

In [ ]:
# Show the amount of produced items per month for the agent
a.plot_produced_items()

In [ ]:
# Show the amount of produced items per month from the optimal solution
a.plot_produced_items(a.optimal_solution)

## ExpensiveMaterialAgent
This agent builds the products that have the highest costs first.
It does not make any differences between the costs, they are all recieve the same weight.

In [ ]:
from agents import ExpensiveMaterialAgent

In [ ]:
e = ExpensiveMaterialAgent(model)
e.simulate_year()
agents.append(e)

In [ ]:
# Compare the agent profit with the optimal solution
e.print_profits()

In [ ]:
e.plot_profits()

In [ ]:
e.plot_produced_items()

In [ ]:
e.plot_produced_items(e.optimal_solution)

## NaiveProfitAgent
This agent builds the products that promise the most profit first.
It does not consider any costs.

In [ ]:
from agents import NaiveProfitAgent
c = NaiveProfitAgent(model)
c.simulate_year()
agents.append(c)

In [ ]:
c.print_profits()

In [ ]:
c.plot_profits()

In [ ]:
c.plot_produced_items()

In [ ]:
c.plot_produced_items(c.optimal_solution)

## MarginProfitAgent
This agent behaves the same as the naive profit agent but considers the costs of the items

In [ ]:
from agents import MarginProfitAgent

In [ ]:
margin_agent = MarginProfitAgent(model)
margin_agent.simulate_year()
agents.append(margin_agent)

In [ ]:
margin_agent.print_profits()

In [ ]:
margin_agent.plot_profits()

In [ ]:
margin_agent.plot_produced_items()

In [ ]:
margin_agent.plot_produced_items(margin_agent.optimal_solution)

## Compare Agents

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

In [ ]:
# Compare the profits of each agent
for a in agents:
    plt.plot(range(1,13), a.profits)
plt.legend([a.name for a in agents])
plt.title('Profits of Every Agent')
plt.xticks(range(1,13))
plt.show()

In [ ]:
sums = [sum(a.profits) for a in agents]
df = pd.DataFrame(sums, index=[a.name for a in agents])
df.columns = ['Profit']
display(df)
df.plot(kind='bar', legend=False,width=0.3,figsize=(7,5),rot=1,title='Profits')
plt.show()

In [ ]:
# Compare built Items
for a in agents:
    print(a.name)
    a.plot_produced_items()

In [ ]:
for a in agents:
    print(a.name)
    a.plot_unused_resources()
a.plot_unused_resources(a.optimal_solution)

## Review variation between items built
In this section the variation coefficient is used to compare the items built

In [ ]:
from agents import stats
optimal_solution = RandomAgent(model).optimal_solution
variation_between_items = [stats.get_distributed_variation(optimal_solution, m) for m in range(12)]
variation_each_item = [stats.get_month_variation(optimal_solution, k) for k in optimal_solution.produced_items.keys()]

In [ ]:
variation_between_items

In [ ]:
variation_each_item

In [ ]:
from agents import WeightedMarginAgent
f = WeightedMarginAgent(model)

In [ ]:
f.simulate_year()

In [ ]:
f.print_profits()

In [ ]:
f.plot_produced_items()